In [1]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta
from classes_func import *

In [2]:
# Creating instance
barcelona = Search('Barcelona')

In [3]:
# Inputing city - google alert needs to be cleared manually
barcelona.input_city()

In [7]:
barcelona.open_date_box()

In [8]:
#Select Date_start
barcelona.date_selector('2024-01-27')

click
found in 1
date not found in page: 1
moving to next page


In [9]:
#Select Date_end
barcelona.date_selector('2024-02-04')

click
found in 1
date not found in page: 1
moving to next page


In [10]:
barcelona.search_results()

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

In [11]:
pages = barcelona.result_pages()
print(pages)

21


In [12]:
# Barcelona.scrape_results(pages)
barcelona.scrape_results(21)

next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked
next button clicked


In [15]:
df= pd.DataFrame(barcelona.df)
df.shape

(456, 6)

In [16]:
df

,Hotels,Ratings,Distance,District,Price,Link
0,H La Paloma Adults Only,"7,6","a 0,7 km del centro","Ciutat Vella, Barcelona",€ 1.138,https://www.booking.com/hotel/es/h-la-paloma.e...
1,Sonder Casa Luz,"8,4",a 450 m del centro,"Eixample, Barcelona",€ 1.602,https://www.booking.com/hotel/es/casa-luz-barc...
2,Express by gaiarooms,"6,9","a 1,8 km del centro","Gracia, Barcelona",€ 1.087,https://www.booking.com/hotel/es/hostal-expres...
3,Leevin Student Barcelona,"9,1","a 3,6 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 589,https://www.booking.com/hotel/es/leevin-hostel...
4,Vincci Maritimo,"8,3","a 4,2 km del centro","Sant Martí, Barcelona",€ 1.306,https://www.booking.com/hotel/es/vinccimaritim...
...,...,...,...,...,...,...
451,Hostelle - Women only hostel Barcelona,"8,5","a 6,3 km del centro","Nou Barris, Barcelona",€ 318,https://www.booking.com/hotel/es/hostelle-barc...
452,Guest House Barcelona Bruc,"6,8","a 0,8 km del centro","Eixample, Barcelona",€ 494,https://www.booking.com/hotel/es/guest-house-b...
453,HelloBCN Youth Hostel Barcelona,"8,0","a 1,5 km del centro","Sants-Montjuïc, Barcelona",€ 359,https://www.booking.com/hotel/es/hellobcn-host...
454,Don Moustache Boutique Hostel,"7,5","a 0,7 km del centro","Eixample, Barcelona",€ 474,https://www.booking.com/hotel/es/bon-moustache...


In [17]:
# Adding descriptions

import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import time

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

# Function to scrape the descriptions using Beautiful Soup
def scrape_description(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() 
        # time.sleep(0.5)
    except requests.exceptions.RequestException as e:
        print(f"Error processing {url}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    description_tag = soup.find('p', class_='a53cbfa6de b3efd73f69')

    if description_tag:
        return description_tag.get_text(strip=True)
    else:
        print(f"Description tag not found on the page: {url}")
        return None

# Set the number of concurrent threads (adjust this based on the processing power of your computer)
num_threads = 16

# Create a ThreadPoolExecutor to run operations in parallel
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Use executor.map to apply the scrape_description function to each URL in parallel
    descriptions = []
    for i, description in enumerate(executor.map(scrape_description, df['Link']), start=1):
        descriptions.append(description)
        # Print every 50 link to check the progess of the scraping
        if i % 50 == 0:
            print(f"Scraped {i} links")

# Assign the descriptions to the 'Descriptions' column in the DataFrame
df['Descriptions'] = descriptions

# Print count after all threads have completed
print(f"Scraped {len(descriptions)} links")
print("\nDone!\n")

Scraped 50 links
Scraped 100 links
Scraped 150 links
Scraped 200 links
Scraped 250 links
Scraped 300 links
Scraped 350 links
Scraped 400 links
Scraped 450 links
Scraped 456 links

Done!



In [18]:
df

,Hotels,Ratings,Distance,District,Price,Link,Descriptions
0,H La Paloma Adults Only,"7,6","a 0,7 km del centro","Ciutat Vella, Barcelona",€ 1.138,https://www.booking.com/hotel/es/h-la-paloma.e...,The H La Paloma is located in the Raval neighb...
1,Sonder Casa Luz,"8,4",a 450 m del centro,"Eixample, Barcelona",€ 1.602,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
2,Express by gaiarooms,"6,9","a 1,8 km del centro","Gracia, Barcelona",€ 1.087,https://www.booking.com/hotel/es/hostal-expres...,El Express by gaiarooms se encuentra en el cén...
3,Leevin Student Barcelona,"9,1","a 3,6 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 589,https://www.booking.com/hotel/es/leevin-hostel...,"Leevin Student Barcelona, en Barcelona, ofrece..."
4,Vincci Maritimo,"8,3","a 4,2 km del centro","Sant Martí, Barcelona",€ 1.306,https://www.booking.com/hotel/es/vinccimaritim...,El Vincci Marítimo se encuentra a 10 minutos a...
...,...,...,...,...,...,...,...
451,Hostelle - Women only hostel Barcelona,"8,5","a 6,3 km del centro","Nou Barris, Barcelona",€ 318,https://www.booking.com/hotel/es/hostelle-barc...,Hostelle - Women only hostel Barcelona está mu...
452,Guest House Barcelona Bruc,"6,8","a 0,8 km del centro","Eixample, Barcelona",€ 494,https://www.booking.com/hotel/es/guest-house-b...,Guest House Barcelona Bruc se encuentra en el ...
453,HelloBCN Youth Hostel Barcelona,"8,0","a 1,5 km del centro","Sants-Montjuïc, Barcelona",€ 359,https://www.booking.com/hotel/es/hellobcn-host...,El Hellobcn Hostel Barcelona está situado en e...
454,Don Moustache Boutique Hostel,"7,5","a 0,7 km del centro","Eixample, Barcelona",€ 474,https://www.booking.com/hotel/es/bon-moustache...,El Don Moustache Hostel goza de una ubicación ...


In [12]:
barcelona.browser.quit()